In [1]:
## 1: Visualizing the displacement profile in the z-direction
plt.plot(u_d.x.array[2::3])
plt.title("Displacement in z-direction")
plt.xlabel("Node Index")
plt.ylabel("Displacement (z-direction)")
plt.grid()
plt.show()

Identify contact areas and plot them
contact_nodes = np.where(u_d.x.array[2::3] >= z1)[0]  # Nodes in contact with the surface at z1
contact_coordinates = pad_node_coordinates[contact_nodes]  # Get the coordinates of contact nodes

Plotting contact nodes
plt.scatter(contact_coordinates[:, 0], contact_coordinates[:, 1], label="Contact Nodes", c='red', s=10)
plt.title("Contact Area on Brake Pad")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.grid()
plt.legend()
plt.show()

SyntaxError: invalid syntax (871854697.py, line 9)

In [ ]:
## 2: Cut the nodes below the z1, but this method is not perfect since only x[2]=z1 in the begin are cuted.

def bottom_contact_nodes(x):
    return np.isclose(x[2], z1)
contact_dofs = fem.locate_dofs_geometrical(Vu, bottom_contact_nodes)
u_d = fem.Function(Vu, name="Displacement")
problem = fem.petsc.LinearProblem(aM, LM, u=u_d, bcs=bcu)
problem.solve()
u_vals = u_d.x.array.reshape(-1, gdim)   # Get displacement values
for dof in contact_dofs:
    if u_vals[dof][2] < z1:
        u_vals[dof][2] = 0  # Set minimum z value to z1
u_d.x.array[:] = u_vals.ravel()
scale_factor = 1
plot_s_pad = plot_S_pad(Vu, u_d, scale_factor)
plot_s_pad.show()

In [ ]:
## 3: Relationship between penalty stiffness and the maximum penatration.

n = 4
deform_max=[]
for i in range(n):

  penalty_param = 400+i*10
  u_d = fem.Function(Vu, name="Displacement")
  problem = fem.petsc.LinearProblem(aM, LM, u=u_d, bcs=bcu)
  problem.solve()
  update_penalty_force(u_d, penalty_forces, z1, penalty_param)
    # Define the modified right-hand side with penalty forces included
  u_d = fem.Function(Vu, name="Displacement")
  LM_penalized = LM + ufl.inner(penalty_forces, u_) * ufl.dx
    # Re-solve with updated forces
  problem = fem.petsc.LinearProblem(aM, LM_penalized, u=u_d, bcs=bcu)
  problem.solve()
  u_d_vals = u_d.x.array.reshape(-1, gdim)  
  
  original_coordinates = domain_pad.geometry.x  # Initial coordinates of nodes (N x 3 array)
  u_d_vals = u_d.x.array.reshape(-1, gdim)  # Displacements (N x 3 array)
  deformed_coordinates = original_coordinates + u_d_vals  # Element-wise addition
  below_z1_nodes = deformed_coordinates[ deformed_coordinates[:, 2] <= z1]  # Select nodes with z < z1
  close_z1_nodes = deformed_coordinates  [ ( deformed_coordinates[:, 2] < 21) & (deformed_coordinates[:, 2] > 19) ] 
  deform_max.append(min( below_z1_nodes[:,2]))
c = list( range(1,n+1))
plt.plot(c, deform_max)


In [ ]:
## 4: Get facet nodes connection
target_tag = 18  # Replace with the actual tag of interest

nodes = domain_pad.geometry.x

facet_dim = domain_pad.topology.dim - 1  # For 3D, facet_dim is 2
domain_pad.topology.create_connectivity(facet_dim, 0)  # Facet to vertex connectivity
facet_to_vertex = domain_pad.topology.connectivity(facet_dim, 0)

i=0

for facet_index in range(len(facet_mark_pad.values)):

    if facet_mark_pad.values[facet_index] == target_tag:
      
        node_indices = facet_to_vertex.links(facet_index)      
      
        facet_nodes = nodes[node_indices]    
        i = i+1
        
        print(f"Facet {facet_index} with tag {target_tag} has nodes {node_indices}")


In [ ]:
## 5: Another way to found the contact nodes for original mesh
len_dofs = 0
for j in range(18):
    marker = (j + 1) * 10
    locator_func1 = lambda x, j=j: (x[0] - x_co[j])**2 + (x[1] - y_co[j])**2 <= S_rub_circle  
    dofs1 = fem.locate_dofs_geometrical(Vu, locator_func1)
    locator_func2 = lambda x : np.isclose(x[2],z1) 
    # np.isclose is betwen than x[2]=z1, since it accounting for potential floating error,
    # for x[2]=z1, it have exactly number for this, for mesh, it generate error.
    dofs2 = fem.locate_dofs_geometrical(Vu, locator_func2)
    dofs3 = np.intersect1d(dofs1, dofs2)
    len_dofs = len_dofs+ len(dofs3)   
    #boundary_dofs.append((marker, dofs3))
    boundary_dofs.append(dofs3)
dof_coordinates = Vu.tabulate_dof_coordinates().reshape(-1, 3)  # Auto-detect dimensions
!# Extract nodes corresponding to these dofs for each boundary marker
dofs4 = []
!#for marker, dofs in boundary_dofs:
for dofs in boundary_dofs:
    boundary_nodes = dof_coordinates[dofs]
    dofs4 = np.hstack( (dofs4, dofs) )
dofs4 = dofs4.astype(int)    
!#dofs4 = np.sort(dofs4)
    
print ('1: Len of boundary nodes ', len(boundary_nodes)  ) 
print('2: Len of dof coordinates ',len(dof_coordinates))
print('3: Len of contact nodes ', len(dofs4))

print(boundary_dofs[17])

In [ ]:
## 6: Compare the length for old and new contact
this loop is used to get the length of the rub contact nodes
facet_1 = []
rub_c_nodes = []
for i in range(18):
    define_i = lambda x: (x[0]-x_co[i])**2 + (x[1]-y_co[i])**2 <= S_rub_circle 
    dofs1 = fem.locate_dofs_geometrical(Vu, define_i)
    # np.isclose is betwen than x[2]=z1, since it accounting for potential floating error,
    # for x[2]=z1, it have exactly number for this, for mesh, it generate error.
    define_contact_s = lambda x: np.isclose(x[2],z1)
    dofs2 = fem.locate_dofs_geometrical(Vu, define_contact_s)
    dofs3 = np.intersect1d(dofs1, dofs2)
    dofs3 = dofs3.astype(int)
    rub_c_nodes = rub_c_nodes+ [dofs3]


length_c_nodes = 0
for i in range( len( rub_c_nodes)):
    length_c_nodes += len( rub_c_nodes[i])
print('1: Old contact nodes number is ',length_c_nodes)
n_contact_new = sum( len(item) for item in new_c)
print("2: New contact nodes number is ", n_contact_new)

old_c_nodes = rub_c_nodes 